In [2]:
######## Test!!!!

In [3]:
import numpy as np,\
pandas as pd,\
seaborn as sns,\
matplotlib.pyplot as plt,\
sklearn.preprocessing as Scaler

import sklearn

from scipy import stats

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [4]:
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')

In [5]:
features_to_drop = ['GarageArea', 'TotRmsAbvGrd', 'TotalBsmtSF']
features_to_trans = ['MiscVal', 'PoolArea', 'LotArea', '3SsnPorch', 'LowQualFinSF',
                     'KitchenAbvGr', 'BsmtFinSF2', 'ScreenPorch', 'BsmtHalfBath',
                     'EnclosedPorch', 'MasVnrArea', 'OpenPorchSF', 'LotFrontage',
                     'BsmtFinSF1', 'WoodDeckSF', '1stFlrSF',
                     'GrLivArea', 'BsmtUnfSF', '2ndFlrSF', 'HalfBath']
features_categorical = ['LotShape', 'OverallQual', 'OverallCond', 'LandContour', 'YearBuilt', 'Utilities', 'LotConfig',
                        'YearRemodAdd', 'GarageFinish', 'MoSold', 'GarageQual', 'YrSold', 'GarageCond', 'PavedDrive',
                        'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition', 'MSZoning', 'MSSubClass',
                        'Street', 'Alley', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
                        'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
                        'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                        'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu',
                        'GarageType', 'GarageYrBlt']
features_numerical = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', '1stFlrSF',
                      '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
                      'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces', 'GarageCars', 'WoodDeckSF',
                      'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']

In [6]:
# Join training and test dataframes for easy preprocessing
#
######################################
Y = train_data[['Id', 'SalePrice']]
all_data = pd.concat((train_data.drop(columns = ['SalePrice']), test_data))

In [7]:
# Get and Split N/A faetures
#
######################################
features_na = (all_data.isnull() == True).sum().sort_values(ascending = False)
features_na = features_na[features_na > 0].index
features_na_num = [x for x in features_na if x in features_numerical]
features_na_cat = [x for x in features_na if x in features_categorical]

In [8]:
# Fill NA
#
######################################
all_data = all_data.drop(columns = features_to_drop)

all_data['MSZoning'].fillna('RL')
all_data['Functional'].fillna('Typ')
all_data['Utilities'].fillna('AllPub')
all_data['Electrical'].fillna('SBrkr')
all_data['SaleType'].fillna('WD')
all_data['KitchenQual'].fillna('TA')
all_data['Exterior1st'].fillna('VinylSd')
all_data['Exterior2nd'].fillna('VinylSd')

all_data[features_na_cat] = all_data[features_na_cat].fillna("None")

all_data['LotFrontage'] = all_data.groupby("Neighborhood")["LotFrontage"].transform(lambda col: 
                                                                                    col.fillna(col.dropna().median()))

all_data[features_na_num] = all_data[features_na_num].fillna(0)


In [9]:
# Log Transformation
#
#######################################
Y['SalePrice'] = np.log1p(Y['SalePrice'])

all_data[features_to_trans] = np.log1p(all_data[features_to_trans])

In [10]:
# Convert Fake numerical Features to strings
# 
#######################################
all_data[['OverallQual', 'OverallCond', 'YearBuilt', 'MSSubClass']] = all_data[
    ['OverallQual', 'OverallCond', 'YearBuilt', 'MSSubClass']].astype(str)

In [15]:
# assign dummy variable
#
#######################################

all_data_dum = pd.get_dummies(all_data.drop('Id', axis = 1))
all_data_dum

,LotFrontage,LotArea,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,LowQualFinSF,...,SaleType_New,SaleType_None,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,4.189655,9.042040,2003,5.283204,6.561031,0.000000,5.017280,6.753438,6.751101,0.0,...,0,0,0,1,0,0,0,0,1,0
1,4.394449,9.169623,1976,0.000000,6.886532,0.000000,5.652489,7.141245,0.000000,0.0,...,0,0,0,1,0,0,0,0,1,0
2,4.234107,9.328212,2002,5.093750,6.188264,0.000000,6.075346,6.825460,6.765039,0.0,...,0,0,0,1,0,0,0,0,1,0
3,4.110874,9.164401,1970,0.000000,5.379897,0.000000,6.293419,6.869014,6.629363,0.0,...,0,0,0,1,1,0,0,0,0,0
4,4.442651,9.565284,2000,5.860786,6.486161,0.000000,6.196444,7.044033,6.960348,0.0,...,0,0,0,1,0,0,0,0,1,0
5,4.454347,9.555064,1995,0.000000,6.597146,0.000000,4.174387,6.680855,6.340359,0.0,...,0,0,0,1,0,0,0,0,1,0
6,4.330733,9.218804,2005,5.231109,7.222566,0.000000,5.762051,7.435438,0.000000,0.0,...,0,0,0,1,0,0,0,0,1,0
7,4.394449,9.247925,1973,5.484797,6.756932,3.496508,5.379897,7.010312,6.891626,0.0,...,0,0,0,1,0,0,0,0,1,0
8,3.951244,8.719481,1950,0.000000,0.000000,0.000000,6.859615,6.930495,6.624065,0.0,...,0,0,0,1,1,0,0,0,0,0
9,3.931826,8.912069,1950,0.000000,6.747587,0.000000,4.948760,6.982863,0.000000,0.0,...,0,0,0,1,0,0,0,0,1,0
